# Load Data : 

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BinancePipeline") \
    .getOrCreate()



Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/19 16:25:39 WARN Utils: Your hostname, DESKTOP-Q0IAP8C, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/19 16:25:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 16:25:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:

bronze_path = '/mnt/c/Users/user/Desktop/Quant-AI-Project/ml/data/btc_minute_data.parquet'
df_bronze = spark.read.parquet(bronze_path)

df_bronze.show(5)
df_bronze.printSchema()

+-------------------+--------+--------+--------+--------+--------+--------------------+------------------+----------------+---------------------+----------------------+------+
|          open_time|    open|    high|     low|   close|  volume|          close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+-------------------+--------+--------+--------+--------+--------+--------------------+------------------+----------------+---------------------+----------------------+------+
|2026-01-19 05:24:00| 92628.0|92634.77|92595.62|92613.99|11.19022|2026-01-19 05:24:...|   1036457.1385852|            2100|              4.05786|        375821.0803997|     0|
|2026-01-19 05:25:00|92613.99|92613.99|92613.98|92613.99| 1.81514|2026-01-19 05:25:...|    168107.3438968|             196|              0.42396|         39264.6272004|     0|
|2026-01-19 05:26:00|92613.99|92627.42|92613.98|92627.42|  5.1339|2026-01-19 05:26:...|    475507.0662718|             7

In [3]:
df_bronze.summary().show()

26/01/19 16:29:02 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------------+----------------------+------+
|summary|              open|              high|               low|             close|           volume|quote_asset_volume|  number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+-------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------------+----------------------+------+
|  count|               600|               600|               600|               600|              600|               600|               600|                  600|                   600|   600|
|   mean| 92918.16596666667|       92934.30065| 92903.02651666672| 92918.76260000002| 7.64264591666666| 710231.6010940704|           1791.86|   3.6437998333333317|    338615.74439106707|   0.0|
| stddev|218.01073153671751|21

In [5]:
df_bronze.printSchema()

root
 |-- open_time: timestamp_ntz (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- close_time: timestamp_ntz (nullable = true)
 |-- quote_asset_volume: double (nullable = true)
 |-- number_of_trades: long (nullable = true)
 |-- taker_buy_base_volume: double (nullable = true)
 |-- taker_buy_quote_volume: double (nullable = true)
 |-- ignore: string (nullable = true)



In [4]:
# Check for nulls
from pyspark.sql.functions import col, sum as spark_sum, when

null_counts = df_bronze.select([
    spark_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) 
    for c in df_bronze.columns
])
null_counts.show()

+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------+----------------------+------+
|open_time|open|high|low|close|volume|close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------+----------------------+------+
|        0|   0|   0|  0|    0|     0|         0|                 0|               0|                    0|                     0|     0|
+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------+----------------------+------+



In [8]:
print("Déduplication...")
rows_before = df_bronze.count()
df_bronze = df_bronze.dropDuplicates(["open_time"])
duplicates = rows_before - df_bronze.count()
print(f" {duplicates} doublons supprimés")

Déduplication...
 0 doublons supprimés
